In [1]:
import json

with open("/kaggle/input/dataset/formatted_coqar_data.json", "r") as f:
    data = json.load(f)

In [2]:
data.keys()

dict_keys(['input', 'references', 'context', 'answer_spans'])

In [3]:
data['input'][1]

['What color was Cotton?', 'white', 'Where did she live?']

In [4]:
from datasets import Dataset

x = Dataset.from_dict(data)

In [5]:
x[2]

{'input': ['What color was Cotton?',
  'white',
  'Where did she live?',
  'in a barn',
  'Did she live alone?'],
 'references': ['Did Cotton live all alone?',
  'Was Cotton alone in the barn?',
  'Did Cotton live all by herself?'],
 'context': 'Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer\'s horses slept. But Cotton wasn\'t alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton\'s mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer\'s orange paint, she used it to paint herself like them. When her mommy and sisters found

In [6]:
#context + input -> references

In [7]:
#context question rewriting

In [8]:
new_data = []
for i in x:
    if len(i['input']) > 1:
        count = 0
        for ii in range(len(i['input'])-1):
            if ii %2 == 0:
                i['context'] += f"\nquestion {count +1}: {i['input'][ii]}"
#                 i['context'] += f"\n{i['input'][ii]}"
            else:
                i['context'] += f"\nanswer {count +1}: {i['input'][ii]}"
#                 i['context'] += f"\n {i['input'][ii]}"
                count += 1
                
        i['input'] = i['input'][-1]
    else:
        i['input'] = i['input'][0]
    for ii in i['references']:
        new_data.append({"context": i['context'], "question": i['input'], "rewriting": ii})

In [9]:
new_data

[{'context': 'Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer\'s horses slept. But Cotton wasn\'t alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton\'s mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer\'s orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n"What are you doing, Cotton?!" \n\n"I only wanted to be more like you". \n\nCotton\'s mommy rubbed her face on Cotton\'s and said "Oh Cotton, but your fur is so pretty and special, like you. We would 

In [10]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-b

In [11]:
data_dict = {key: [d[key] for d in new_data] for key in new_data[0]}

In [12]:
o = Dataset.from_dict(data_dict)

In [13]:
o

Dataset({
    features: ['context', 'question', 'rewriting'],
    num_rows: 101403
})

In [14]:
!pip install datasets

In [15]:
from datasets import load_dataset
from datasets import Dataset

In [16]:
#data = Dataset.from_pandas(data)

In [17]:
data = list(data)

In [18]:
o

Dataset({
    features: ['context', 'question', 'rewriting'],
    num_rows: 101403
})

In [19]:
train_size = len(o) * 80 // 100
dev_size = len(o) * 10 // 100
test_size = len(o) - dev_size - train_size

train = o[:train_size]
dev = o[train_size:train_size + dev_size]
test = o[train_size + dev_size:]

print(len(train))
print(len(dev))
print(len(test))


3
3
3


In [20]:
import pandas as pd

In [21]:
train_df = pd.DataFrame(train)
dev_df = pd.DataFrame(dev)
test_df = pd.DataFrame(test)

In [22]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
# model_inputs = tokenizer(
#     pad_on_right = tokenizer.padding_side == "right",
#     "Hủy hoại di vật tử sĩ đi tù bao nhiêu năm?", 
#     "Đi tù từ 3 năm đến 7 năm", 
#     truncation="only_second" if pad_on_right else "only_first",
#     max_length=1024,
#     padding="max_length"
# )


In [25]:
#tokenizer.decode(model_inputs['input_ids'])

In [26]:
def preprocess_function(examples):
    pad_on_right = tokenizer.padding_side == "right"
    model_inputs = tokenizer(
        examples["question" if pad_on_right else "context"], #
        examples["context" if pad_on_right else "question"], #
        truncation="only_second" if pad_on_right else "only_first",
        max_length=1024,
        padding="max_length")
    labels = tokenizer(
        examples["rewriting"], max_length=512, truncation=True, padding=True #
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [27]:
train_dataset

Dataset({
    features: ['context', 'question', 'rewriting'],
    num_rows: 81122
})

In [28]:
train_set = train_dataset.map(preprocess_function, batched=True, remove_columns=['context', 'question', 'rewriting'])

  0%|          | 0/82 [00:00<?, ?ba/s]

In [29]:
dev_set = dev_dataset.map(preprocess_function, batched=True, remove_columns = ['context', 'question', 'rewriting'])

  0%|          | 0/11 [00:00<?, ?ba/s]

In [30]:
test_set = test_dataset.map(preprocess_function, batched=True, remove_columns =['context', 'question', 'rewriting'])

  0%|          | 0/11 [00:00<?, ?ba/s]

In [31]:
from transformers import AutoModelForSeq2SeqLM

# Tải mô hình T5 từ Hub
model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [32]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 1

args = Seq2SeqTrainingArguments(
    output_dir=f"T5-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
)

In [33]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

2024-06-02 03:16:57.237707: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 03:16:57.237813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 03:16:57.356208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [34]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=585ae2b3669d36b9a5856eb2fcd930cc5c76267fa04aa800120c9a432b0f4092
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [35]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [36]:
import evaluate

rouge_score = evaluate.load("rouge")

In [37]:
!pip install nltk

In [38]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from nltk.tokenize import sent_tokenize

In [40]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return {k: round(v, 4) for k, v in result.items()}

In [41]:

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_set,
    eval_dataset=dev_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
import wandb
wandb.login(key = 'F68567BCA11984E5C39F239F101B619C1344A0F6')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
trainer.train()

wandb: Currently logged in as: khangcuuthi (velaciela). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def bleu(predict, goal):
    bleu_scores = {1: [], 2: [], 3: [], 4: []}

    for sent1, sent2 in zip(predict, goal):
        ws = [(1, 0, 0, 0), (0.5, 0.5, 0, 0), (0.33, 0.33, 0.33, 0), (0.25, 0.25, 0.25, 0.25)]
        for n in range(1, 5):
            weights = ws[n-1]
            sent1_tokens = sent1.split()
            sent2_tokens = sent2.split()
            bleu_score = sentence_bleu([sent1_tokens], sent2_tokens, weights=weights)
            bleu_scores[n].append(bleu_score)
    result = {}
    for n in range(1, 5):
        avg_bleu_score = (sum(bleu_scores[n]) / len(bleu_scores[n]))*100
        result["BLEU{}".format(n)] = [(sum(bleu_scores[n]) / len(bleu_scores[n]))*100]
    return result

In [ ]:
import torch
from tqdm.notebook import tqdm

def gen(tokenized_test):
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
    
    max_target_length = 50
    dataloader = torch.utils.data.DataLoader(tokenized_test, collate_fn=data_collator, batch_size=16)

    predictions = []
    references = []
    for i, batch in enumerate(tqdm(dataloader)):
        outputs = model.generate(
          input_ids=batch['input_ids'].to('cuda'),
          max_length=max_target_length,
          attention_mask=batch['attention_mask'].to('cuda'),
        )
        with tokenizer.as_target_tokenizer():
            outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]
            labels = np.where(batch['labels'] != -100, batch['labels'], tokenizer.pad_token_id)
            actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
        
        predictions.extend(outputs)
        references.extend(actuals)
    
    predictions = [o for o in predictions]
    references = [o for o in references]
    pd.DataFrame({"predictions": predictions, "references": references}).to_csv("output.csv", index = False, encoding = 'utf-8')
    return predictions, references

In [ ]:
output, ref = gen(test_set)


In [ ]:
output

In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")
rouge = rouge_metric.compute(predictions=output, references=ref)
result = {'rouge1': [rouge["rouge1"].mid.fmeasure*100],
         'rouge2': [rouge["rouge2"].mid.fmeasure*100],
         'rougeL': [rouge["rougeL"].mid.fmeasure*100]}
result1 = {**result, **bleu(output, ref)}

In [ ]:
result1

In [ ]:
# !pip install -U nltk
# pip install datasets

In [ ]:
# import nltk
# from datasets import load_metric
# from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# # Download necessary nltk data
# nltk.download('punkt')
# nltk.download('wordnet')

# # Load ROUGE metric
# rouge_metric = load_metric("rouge")

# # Sample predictions and references
# predictions = ["This is a test prediction", "Another test prediction"]
# references = ["This is a test reference", "Another reference for testing"]

# # Calculate BLEU score
# def calculate_bleu(predictions, references):
#     tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]
#     tokenized_references = [[nltk.word_tokenize(ref)] for ref in references]
    
#     smoothing_function = SmoothingFunction().method4
#     bleu_score = corpus_bleu(tokenized_references, tokenized_predictions, smoothing_function=smoothing_function)
    
#     return bleu_score

# # Calculate ROUGE score
# def calculate_rouge(predictions, references):
#     rouge_results = rouge_metric.compute(predictions=predictions, references=references)
#     return rouge_results

# # Calculate METEOR score
# def calculate_meteor(predictions, references):
#     scores = []
#     for pred, ref in zip(predictions, references):
#         score = nltk.translate.meteor_score.single_meteor_score(ref, pred)
#         scores.append(score)
#     meteor_avg = sum(scores) / len(scores)
#     return meteor_avg

# # Example usage
# bleu = calculate_bleu(predictions, references)
# rouge = calculate_rouge(predictions, references)
# meteor = calculate_meteor(predictions, references)

# print("BLEU score:", bleu)
# print("ROUGE scores:", rouge)
# print("METEOR score:", meteor)


In [ ]:
# import nltk
# from datasets import load_metric
# from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# def calculate_meteor(predictions, references):
#     scores = []
#     for pred, ref in zip(predictions, references):
#         score = nltk.translate.meteor_score.single_meteor_score(ref, pred)
#         scores.append(score)
#     meteor_avg = sum(scores) / len(scores)
#     return meteor_avg